In [1]:
!pip install streamlit langchain duckduckgo-search beautifulsoup4 fuzzywuzzy python-Levenshtein transformers

In [2]:
import importlib

packages = [
    "streamlit",
    "langchain",
    "duckduckgo_search",
    "bs4",
    "fuzzywuzzy",
    "Levenshtein",
    "transformers"
]

for pkg in packages:
    found = importlib.util.find_spec(pkg) is not None
    print(f"{pkg}: {'✅ Installed' if found else '❌ Not Installed'}")


streamlit: ✅ Installed
langchain: ✅ Installed
duckduckgo_search: ✅ Installed
bs4: ✅ Installed
fuzzywuzzy: ✅ Installed
Levenshtein: ✅ Installed
transformers: ✅ Installed


In [6]:
%%writefile app.py
import streamlit as st
from fuzzywuzzy import process
from duckduckgo_search import DDGS
from transformers import pipeline
import time

st.title("NEWSGenie: a smart company news chatbot")

KNOWN_COMPANIES = [
    "Adobe", "Airbnb", "Amazon", "AMD", "Apple", "Atlassian", "Cisco", "Dell",
    "DoorDash", "Dropbox", "eBay", "Facebook", "Google", "HP", "IBM", "Intel",
    "LinkedIn", "Lyft", "Mastercard", "Meta", "Microsoft", "Netflix", "NVIDIA",
    "Oracle", "PayPal", "Pinterest", "Qualcomm", "Salesforce", "Samsung",
    "Shopify", "Slack", "Snap Inc", "Snapchat", "Sony", "Spotify", "Square",
    "Stripe", "Tesla", "TikTok", "Twitter", "Uber", "Visa", "Zoom"
]

summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

if "conversations" not in st.session_state:
    st.session_state.conversations = []
if "selected_conv" not in st.session_state:
    st.session_state.selected_conv = None
if "feedback" not in st.session_state:
    st.session_state.feedback = {}
if "last_user_input" not in st.session_state:
    st.session_state.last_user_input = ""

def detect_intent(message):
    greeting_keywords = ['hi', 'hello', 'hey', 'greetings']
    if any(word in message.lower() for word in greeting_keywords):
        return 'greeting'
    if message.strip():
        return 'company_query'
    return 'other'

def correct_company_name(name):
    if not KNOWN_COMPANIES:
        return None
    match, score = process.extractOne(name, KNOWN_COMPANIES)
    if score > 80:
        return match
    return None

def get_company_names(user_message):
    company_names = []
    if KNOWN_COMPANIES:
        for word in user_message.split():
            corrected = correct_company_name(word)
            if corrected and corrected not in company_names:
                company_names.append(corrected)
    if not company_names:
        company_names.append(user_message.strip())
    return company_names

def get_news(company, max_results=3):
    with DDGS() as ddgs:
        time.sleep(1)
        results = ddgs.news(company, max_results=max_results)
        news_items = []
        for r in results:
            title = r.get('title')
            url = r.get('url')
            snippet = r.get('body') or ''
            news_items.append((title, url, snippet))
        return news_items

def summarize_news(news_items, style):
    text = "\n".join([f"{t}: {s}" for t, _, s in news_items if t and s])
    if not text.strip():
        return "No details to summarize."
    style_lower = style.lower()
    if "formal" in style_lower:
        prompt = f"Formal business summary:\n{text}"
        summary = summarizer(prompt[:1024], max_length=150, min_length=80, do_sample=False)
        return summary[0]['summary_text']
    elif "casual" in style_lower:
        prompt = f"Friendly, conversational summary:\n{text}"
        summary = summarizer(prompt[:1024], max_length=150, min_length=80, do_sample=True)
        return summary[0]['summary_text']
    elif "bullet" in style_lower:
        prompt = f"Summarize as at least 5 bullet points:\n{text}"
        result = summarizer(prompt[:1024], max_length=100, min_length=50, do_sample=False)[0]["summary_text"]
        raw_bullets = [b.strip(" -.") for b in result.split('\n') if b.strip().startswith("-")]
        if len(raw_bullets) < 4:
            raw_bullets = [b.strip(" -.") for b in result.split('-') if b.strip()]
        bullets = [b for i, b in enumerate(raw_bullets) if b and (b not in raw_bullets[:i])]
        return "\n".join([f"- {b}" for b in bullets])
    else:
        prompt = f"Summary:\n{text}"
        summary = summarizer(prompt[:1024], max_length=150, min_length=80, do_sample=False)
        return summary[0]['summary_text']

user_input = st.text_input("Type company names or chat here...")

style = st.selectbox("Choose output style:", [
    "Formal business summary",
    "Casual conversation",
    "Quick bullet points"
])

# Process new input only when changed
if user_input and user_input != st.session_state.last_user_input:
    intent = detect_intent(user_input)
    if intent == "greeting":
        bot_reply = "Hi there! You can enter any company name and I'll find the latest news for you."
    elif intent == "company_query":
        company_names = get_company_names(user_input)
        bot_reply = ""
        for company_name in company_names:
            news = get_news(company_name)
            if news:
                summary = summarize_news(news, style)
                bot_reply += f"News for {company_name}:\n{summary}\n\n"
                for idx, (title, url, _) in enumerate(news, 1):
                    bot_reply += f"{idx}. [{title}]({url})\n"
                bot_reply += "\n---\n"
            else:
                bot_reply += f"Sorry, I couldn't find any recent news for {company_name}.\n---\n"
    else:
        bot_reply = "Sorry, I didn't understand that. Please enter a company name."

    st.session_state.conversations.append({"user": user_input, "bot": bot_reply})
    st.session_state.selected_conv = len(st.session_state.conversations) - 1
    st.session_state.last_user_input = user_input

with st.sidebar:
    st.markdown("Recent Chats")
    for idx in reversed(range(len(st.session_state.conversations))):
        conv = st.session_state.conversations[idx]
        if st.button(conv["user"][:60], key=f"selectbtn_{idx}"):
            st.session_state.selected_conv = idx

selected_idx = st.session_state.get("selected_conv", None)
if selected_idx is not None and selected_idx < len(st.session_state.conversations):
    conv = st.session_state.conversations[selected_idx]
    st.markdown(f"**You:** {conv['user']}")
    st.markdown(f"**Bot:** {conv['bot']}")
    col1, col2 = st.columns([1, 1])
    with col1:
        if st.button("👍", key=f"up_{selected_idx}"):
            st.session_state.feedback[selected_idx] = "up"
    with col2:
        if st.button("👎", key=f"down_{selected_idx}"):
            st.session_state.feedback[selected_idx] = "down"
    fb = st.session_state.feedback.get(selected_idx)
    if fb == "up":
        st.success("Thank you for your positive feedback! 👍")
    elif fb == "down":
        st.error("Thanks for your feedback. 👎")
elif st.session_state.conversations:
    conv = st.session_state.conversations[-1]
    st.markdown(f"**You:** {conv['user']}")
    st.markdown(f"**Bot:** {conv['bot']}")
else:
    st.markdown("_No chat history yet_")


Writing app.py


In [7]:
from pyngrok import ngrok

ngrok.set_auth_token('30rKPi0egXAHDJYiIoG6jD2R3Mn_twNK1UNpL8UCnj3W7RkA')


In [4]:
import os
os.system('killall streamlit')  # To avoid port conflicts
os.system('nohup streamlit run app.py &')
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at: {public_url}")

Streamlit app is live at: NgrokTunnel: "https://39e5a1a9c5bb.ngrok-free.app" -> "http://localhost:8501"
